In [ ]:
!pip3 install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=1749a896d3a8ca235a5db12c84b80a9899a8c660bfdddca35c26483af5b53ebd
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# Import libraries

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


# Create SparkSession
spark = SparkSession.builder.appName('Song_Review_Analysis') \
        .master("local[8]").getOrCreate()

# Import Dataset

In [ ]:
!unzip dataset.zip

unzip:  cannot find or open dataset.zip, dataset.zip.zip or dataset.zip.ZIP.


In [ ]:
data = spark.read.csv("songsDataset.csv",header=True)
data

DataFrame['userID': string, 'songID': string, 'rating': string]

In [ ]:
data = data.select(col("'userID'").alias("userID"), col("'songID'").alias("songID"), col("'rating'").alias("rating"))

# High Level Overview of Dataset

In [ ]:
review_count_df = data.groupBy("userID").count()

In [ ]:
review_count_df.show()

+------+-----+
|userID|count|
+------+-----+
|   296|   10|
|   467|   10|
|   675|   10|
|   691|   10|
|   829|   10|
|  1090|   10|
|  1159|   10|
|  1436|   10|
|  1512|   10|
|  1572|   10|
|  2069|   10|
|  2088|   10|
|  2136|   10|
|  2162|   10|
|  2294|   10|
|  2904|   10|
|  3210|   10|
|  3414|   10|
|  3606|   10|
|  3959|   10|
+------+-----+
only showing top 20 rows



In [ ]:
review_count_df.filter(review_count_df.userID == 199999).show()

+------+-----+
|userID|count|
+------+-----+
|199999|   10|
+------+-----+



# Positive Interaction Filtering

In [ ]:
positive_rating_data = data.filter(data.rating >= 3)

positive_rating_data.count()

1412937

In [ ]:
positive_review_count_df = positive_rating_data.groupBy("userID").count()

positive_review_count_df = positive_review_count_df.select(col("userID"), col("count").alias("review_count"))

positive_review_count_df.show()

+------+------------+
|userID|review_count|
+------+------------+
|   296|           6|
|   467|           5|
|   675|           6|
|   691|          10|
|   829|           9|
|  1159|           3|
|  1436|           7|
|  1512|           5|
|  1572|           4|
|  2069|           7|
|  2088|           7|
|  2136|           5|
|  2162|           9|
|  2294|           9|
|  2904|           7|
|  3210|           7|
|  3414|          10|
|  3606|          10|
|  3959|           8|
|  4032|           9|
+------+------------+
only showing top 20 rows



In [ ]:
positive_review_count_df.sort("review_count").show()

+------+------------+
|userID|review_count|
+------+------------+
| 29892|           1|
|  4594|           1|
| 22093|           1|
| 23847|           1|
|   233|           1|
| 28695|           1|
| 20274|           1|
| 26863|           1|
|   993|           1|
|  1706|           1|
| 17146|           1|
|   451|           1|
|  3083|           1|
| 11510|           1|
| 19205|           1|
|  3368|           1|
|  8901|           1|
|  3365|           1|
| 25812|           1|
| 15052|           1|
+------+------------+
only showing top 20 rows



# Filter only those users who rated positively at least 5 songs

In [ ]:
active_positive_review_count_df = positive_review_count_df.filter(positive_review_count_df.review_count >= 5)

In [ ]:
final_data = active_positive_review_count_df.join(positive_rating_data, how="left", on="userID")

In [ ]:
final_data.count()

1315321

In [ ]:
final_data.show()

+------+------------+------+------+
|userID|review_count|songID|rating|
+------+------------+------+------+
|  1436|           7| 21703|     5|
|  1436|           7| 73763|     5|
|  1436|           7| 87408|     4|
|  1436|           7|102517|     5|
|  1436|           7|110084|     5|
|  1436|           7|135772|     5|
|  1436|           7|136449|     5|
|  1512|           5| 20711|     5|
|  1512|           5| 33496|     5|
|  1512|           5| 53149|     3|
|  1512|           5|107034|     3|
|  1512|           5|112977|     4|
|  2069|           7| 44119|     5|
|  2069|           7| 59067|     5|
|  2069|           7| 86486|     5|
|  2069|           7| 87658|     5|
|  2069|           7| 92043|     5|
|  2069|           7|104751|     5|
|  2069|           7|120983|     5|
|  2088|           7|  6853|     3|
+------+------------+------+------+
only showing top 20 rows



In [ ]:
final_data.agg({"review_count": "min"}).collect()[0][0]

5

In [ ]:
final_data.agg({"rating": "min"}).collect()[0][0]

'3'

In [ ]:
final_data.write.option("header",True).mode("overwrite").csv("preprocessed_songsDataset.csv")

In [ ]:
selected_user = final_data.select('userID').distinct().collect()[:50000]

selected_user = list(map(lambda x: x['userID'], selected_user))

small_dataset = final_data.filter(final_data.userID.isin(selected_user))

In [ ]:
small_dataset.write.option("header",True).mode("overwrite").csv("sample_songsDataset.csv")

In [ ]:
pandas_df = small_dataset.toPandas()
pandas_df.to_csv('sample_songsDataset.csv')

In [2]:
import pandas as pd

preprocessed_data = pd.read_csv('preprocessed_songsDataset.csv')

pairs_of_user_and_review_count = list(map(lambda x: x[0], list(preprocessed_data.groupby(['userID','review_count']))))

pairs_of_user_and_review_count = sorted(pairs_of_user_and_review_count, key=lambda x: x[1], reverse = True)

selected_user = list(map(lambda x: x[0], pairs_of_user_and_review_count))[:5000]

small_dataset = preprocessed_data[preprocessed_data['userID'].isin(selected_user)]

small_dataset.to_csv("sample_songsDataset.csv")